# Webscraping mit requests

Gegenstand dieses Inputreferetas ist das Webscraping mit der Python-Library `requests`. Die Dokumentation zur Library finden Sie hier: https://docs.python-requests.org/en/master/. 

Wir machen das am Beispiel von Texten der Seite Archive of Our Own (AO3), einer Fanfiction-Seite: https://archiveofourown.org/. 

Die Grundidee ist, eine Suche nach bestimmten Texten nachzubauen; dann die Identifier der relevanten Texte einzusammeln; und dann die Texte in HTML herunterzuladen. 

## Die Arbeitsschritte im Einzelnen

1. Suchabfrage überlegen und im Webinterface vornehmen
2. Suchabfrage mit requests "nachbauen"
3. Liste der Text-IDs einsammeln
4. Alle Texte mit den entsprechenden IDs herunterladen

Mit dem Input aus dem Modul "Auszeichnungssprachen" zur Verarbeitung von HTML mit BeautifulSoup oder lxml (Titel: Markup und Python) können Sie die folgenden Schritte dann auch selbst durchführen:  

5. Metadaten aus den HTML-Dateien extrahieren und abspeichern ("A3O_metadata.tsv")
6.Volltext aus den HTML-Dateien extrahieren und abspeichern. 

Dann ist eine Analyse des Korpus möglich.

## Importe

In [34]:
# Generic imports
from os.path import join
import os

# Specific imports
import re
import requests

## Schritt 1: Die Suchabfrage formulieren

Einstieg: https://archiveofourown.org/works/search. Wir suchen in diesem Beispiel Texte, die den folgenden Kriterien entsprechen: 

- Deutsche Texte
- 10000-20000 Wörter Länge
- Harry Potter-Fandom
- No Crossovers
- No Archive Warnings
- Komplette Texte

Das ergibt im Januar 2022 45 Treffer. Der Link mit den Suchergebnissen ist folgender: https://archiveofourown.org/works/search?utf8=%E2%9C%93&work_search%5Bquery%5D=&work_search%5Btitle%5D=&work_search%5Bcreators%5D=&work_search%5Brevised_at%5D=&work_search%5Bcomplete%5D=T&work_search%5Bcrossover%5D=F&work_search%5Bsingle_chapter%5D=0&work_search%5Bword_count%5D=10000-20000&work_search%5Blanguage_id%5D=de&work_search%5Bfandom_names%5D=Harry+Potter+-+J.+K.+Rowling&work_search%5Brating_ids%5D=&work_search%5Barchive_warning_ids%5D%5B%5D=16&work_search%5Bcharacter_names%5D=&work_search%5Brelationship_names%5D=&work_search%5Bfreeform_names%5D=&work_search%5Bhits%5D=&work_search%5Bkudos_count%5D=&work_search%5Bcomments_count%5D=&work_search%5Bbookmarks_count%5D=&work_search%5Bsort_column%5D=_score&work_search%5Bsort_direction%5D=desc&commit=Search

In diesem Link sind alle Suchkriterien kodiert, sodass wir den Link gut nachbauen und die Suchabfrage auch aus `requests` heraus formulieren können. 

Alle Suchkriterien, die hier keinen Eintrag haben (bei denen auf `=` direkt das `&` folgt, können ignoriert werden. Dadurch vereinfacht sich die Sache deutlich. 

Außerdem erlaubt die Library `requests`, mit der wir das Ganze machen, solche Suchbedingungen als Parameter zu übergeben. Die Library konstruiert dann selbständig eine korrekte Suchanfrage aus dem Basislink der Suche, der Anzahl an Trefferseiten, die berücksichtigt werden sollen, und den Suchkriterien. 

In [36]:
# Parameter

queryurl = "https://archiveofourown.org/works/search?commit=Search"
numpages = 3
querycriteria = {
    "utf8" : "%E2%9C%93",
    "work_search[complete]": "T",
    "work_search[crossover]" : "F",
    "work_search[fandom_names]" : "Harry+Potter+-+J.+K.+Rowling",
    "work_search[language_id]" : "de",
    "work_search[single_chapter]" : "0",
    "work_search[sort_column]" : "_score",
    "work_search[sort_direction]" : "desc",
    "work_search[word_count]" : "10000-20000",
    "work_search[archive_warning_ids][]" : "16"
    }


# Funktionen

def get_html(queryurl, querycriteria): 
    try:
        queryresponse = requests.get(queryurl, params=querycriteria, timeout=4)
        #print(queryresponse.url)
        queryhtml = queryresponse.text
        #print(queryhtml)
        return queryhtml
    except:
        print("Error receiving HTML")
    

def get_ids(queryhtml): 
    try: 
        ids = re.findall(r"<a href=\"/works/(\d*?)\">", queryhtml)
        print(len(ids), ids)
        return ids
    except: 
        print("Error extracting IDs")

            
# Aufruf

all_ids = []
for page in range(1,numpages+1):
    querycriteria["page"] = str(page)
    queryhtml = get_html(queryurl, querycriteria)
    all_ids.extend(get_ids(queryhtml))
print("Total number of ids collected:", len(all_ids))

https://archiveofourown.org/works/search?commit=Search&utf8=%25E2%259C%2593&work_search%5Bcomplete%5D=T&work_search%5Bcrossover%5D=F&work_search%5Bfandom_names%5D=Harry%2BPotter%2B-%2BJ.%2BK.%2BRowling&work_search%5Blanguage_id%5D=de&work_search%5Bsingle_chapter%5D=0&work_search%5Bsort_column%5D=_score&work_search%5Bsort_direction%5D=desc&work_search%5Bword_count%5D=10000-20000&work_search%5Barchive_warning_ids%5D%5B%5D=16&page=1
20 ['27328087', '435741', '31371968', '29333343', '20115061', '655562', '33957355', '11567019', '36319510', '30379272', '1011701', '8377495', '27306145', '11558709', '13808433', '34077589', '11566914', '8142823', '12882969', '8263144']
https://archiveofourown.org/works/search?commit=Search&utf8=%25E2%259C%2593&work_search%5Bcomplete%5D=T&work_search%5Bcrossover%5D=F&work_search%5Bfandom_names%5D=Harry%2BPotter%2B-%2BJ.%2BK.%2BRowling&work_search%5Blanguage_id%5D=de&work_search%5Bsingle_chapter%5D=0&work_search%5Bsort_column%5D=_score&work_search%5Bsort_directi

## Herunterladen der HTML-Dateien nach ID

Jetzt soll für jeden Identifier der entsprechende Text heruntergeladen werden. Hier können wir nun statt mit den Parametern zu arbeiten, den einfacheren Weg gehen, die passende URL sozusagen von Hand zusammenzusetzen. 

In [43]:
# Parameter

A3O_base = "https://archiveofourown.org/"
htmlfolder=join("..", "data", "A3O", "html", "")


# Funktionen
    
def get_html(item): 
    """
    Lade für das jeweilige item (eine ID) die HTML-Datei herunter. 
    Gibt einen String zurück. 
    """
    url = A3O_base + "works/" + item + "?view_full_work=true"
    #print(url)
    response = requests.get(url, timeout=4)
    html = response.text
    #print(html[15000:16000])
    return html


def save_html(item, html):
    """
    Speichere den String in eine HTML-Datei,
    die den Identifier im Dateinamen hat. 
    """
    filename = join(htmlfolder, "A3O-" + str(item) + ".html")
    with open(filename, "w") as outfile: 
        outfile.write(html)
    print("Saved", os.path.basename(filename))


def main(A3O_base, htmlfolder, all_ids): 
    for item in all_ids[0:5]:    # Zum Ausprobieren nur die ersten 5 Texte. 
        html = get_html(item)
        save_html(item, html)

main(A3O_base, htmlfolder, all_ids)

Saved A3O-27328087.html
Saved A3O-435741.html
Saved A3O-31371968.html
Saved A3O-29333343.html
Saved A3O-20115061.html


Das ist auch schon alles! Das neue kleine Korpus ist jetzt auf der Festplatte gespeichert, jeder Text in einer HTML-Datei. Jetzt könnten Sie aus den HTML-Dateien die Metadaten und den Volltext extrahieren, um ein analysierbares Korpus zu erstellen.